In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from __future__ import print_function, division
from builtins import range

In [4]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from gensim.models import KeyedVectors

In [5]:
from nltk.corpus import stopwords


In [6]:
ignore_words = set(stopwords.words('english')).union(set(stopwords.words('indonesian')))
print(len(ignore_words))

936


In [10]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_cat=pd.read_json('categories.json')

In [11]:
df_train.title.head()



0                 nyx sex bomb pallete natural palette
1    etude house precious mineral any cushion pearl...
2                             milani rose powder blush
3                  etude house baby sweet sugar powder
4         bedak revlon color stay aqua mineral make up
Name: title, dtype: object

In [12]:
df_train['Group']=df_train.image_path.map(lambda x: x[:7])
df_test['Group']=df_test.image_path.map(lambda x: x[:7])

In [13]:
train,test=train_test_split(df_train,random_state=2019,stratify=df_train.Category,test_size=0.2)

In [14]:
np.random.seed(2019)

In [15]:
groups=['beauty_','mobile_','fashion']

In [16]:
# some linear models
from sklearn.linear_model import LogisticRegression, BayesianRidge

# SCM for classification
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

import warnings
warnings.filterwarnings('ignore')
np.random.seed(2019)


In [17]:
from sklearn.metrics import accuracy_score

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [19]:
# let's build pipeline for parameter search in n_grams and and various models
lr=LogisticRegression()
nb=MultinomialNB()
svc=SVC()
# let's check first three and see which one is best
gnb=GaussianNB()
bnb=BernoulliNB()
br=BayesianRidge()
rf=RandomForestClassifier(n_estimators=150)

In [25]:
# Best cv scores for parameter for lr.


In [31]:
# We need to ensemble our best results for training. let's get best results from nlp with count vectorization with optimized C 
# parameters as well as min_df
# This are optimization results for C parameter in logistic regression

# params={'model__C':[0.2,0.4,0.8,1,2,4,8]}
# {'beauty_': {'model__C': 0.8},
#  'mobile_': {'model__C': 1},
#  'fashion': {'model__C': 0.4}}

# with the corresponding results
# {'beauty_': 0.78344144628257784,
#  'mobile_': 0.8236574564959771,
#  'fashion': 0.6449463364011252}

# Also did some optimization for min_df the best results for mobile was 2 otherwise 1.

# Let's also check if removing stop words helps to improve model. For the same configuration of optimized parameters

# Submit these results and see if it is really improved in leaderboard
pipe_beauty=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(1,3))),('model',LogisticRegression(C=0.8))])
pipe_mobile=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(1,3),min_df=2)),('model',LogisticRegression(C=1))])
pipe_fashion=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(1,5))),('model',LogisticRegression(C=0.4))])
pipe=[pipe_beauty,pipe_mobile,pipe_fashion]

best_param={}
best_score={}
groups=['beauty_','mobile_','fashion']
param={'vectorize__stop_words':[None,ignore_words]} # to observe effect of stop words
for i in tqdm(range(3)):
    print("Results for ", groups[i])
    grid_search=GridSearchCV(pipe[i],param_grid=param, verbose=1)
    X=df_train[df_train.Group==groups[i]].title
    y=df_train[df_train.Group==groups[i]].Category
    grid_search.fit(X,y)
    best_param[group]=grid_search.best_params_
    print(grid_search.best_params_)
    best_score[group]=grid_search.best_score_
    print(grid_search.best_score_)


## Wow, interesting ! It turns out that stop words helps only for fashion category!
## Lets' list CV results for beauty, mobile  and fashion 
## beauty CV 0.784223069756
## Mobile CV 0.825934011102
## fashion CV 0.644946336401 with stop words

# Results with stop words.
# {'beauty_': 0.78344144628257784,
#  'mobile_': 0.8236574564959771,
#  'fashion': 0.6449463364011252}

# Our base model CV results. I should list them here


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Results for  beauty_
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 17.4min finished


{'vectorize__stop_words': None}
0.784223069756


 33%|███████████████████████████▎                                                      | 1/3 [22:41<45:23, 1361.58s/it]

...

Results for  fashion
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 32.3min finished


{'vectorize__stop_words': {'semisalnya', "you've", 'menurut', 'll', 'diibaratkan', 'nor', 'persoalan', 'sebenarnya', 'terdiri', 'tetapi', 'mengerjakan', 'diungkapkan', 'wouldn', 'kalau', 'each', 'kemudian', 'sendirian', 'on', 'setinggi', 'enggak', 'diri', 'sekalipun', 'juga', 'jelas', 'memihak', 'terakhir', 'antaranya', 'its', 'kalaulah', 'siapa', "aren't", 'terlalu', 'tidak', 'ditanyakan', 'here', 'mendapat', 'dimaksudkannya', 'empat', 'meyakini', 'ucap', 'haven', 'sangat', 'tentang', 'memintakan', 'hingga', 'mana', 'jadilah', 'pantas', 'both', 'some', 'berlebihan', 'perlu', 'sambil', 'katakan', 'semata-mata', 'bagaikan', 'yakni', 'i', 'dia', 'above', 'tambahnya', 'walau', 'disinilah', 'does', 'didn', 'bermacam', 'dini', 'percuma', 'pertanyakan', 'is', 'sesampai', 'pukul', 'himself', 'tentunya', 'belum', 'artinya', 'bahkan', 'para', 'sejumlah', 'wasn', 'lamanya', 'bawah', 'now', 'besar', 'during', 'tadinya', 'sedangkan', 'sebagai', 'hadn', 'namun', 'selalu', 'hal', 'dibuat', 'sayalah'


0.644946336401


100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [1:14:41<00:00, 1547.73s/it]


In [35]:
df_test.head()

,itemid,title,image_path,Group
0,370855998,flormar 7 white cream bb spf 30 40ml,beauty_image/1588591395c5a254bab84042005f2a9f.jpg,beauty_
1,637234604,maybelline clear smooth all in one bb cream sp...,beauty_image/920985ed9587ea20f58686ea74e20f93.jpg,beauty_
2,690282890,murah innisfree eco natural green tea bb cream...,beauty_image/90b40e5710f54352b243fcfb0f5d1d7f.jpg,beauty_
3,930913462,loreal white perfect day cream spf 17 pa white...,beauty_image/289c668ef3d70e1d929d602d52d5d78a.jpg,beauty_
4,1039280071,hada labo cc cream ultimate anti aging spf 35 ...,beauty_image/d5b3e652c5822d2306f4560488ec30c6.jpg,beauty_


In [52]:


# Submit these results and see if it is really improved in leaderboard
pipe_beauty=Pipeline([('vectorize', CountVectorizer(ngram_range=(1,3))),('model',LogisticRegression(C=0.8))])
pipe_mobile=Pipeline([('vectorize', CountVectorizer(ngram_range=(1,3),min_df=2)),('model',LogisticRegression(C=1))])
pipe_fashion=Pipeline([('vectorize', CountVectorizer(stop_words =ignore_words,ngram_range=(1,5))),('model',LogisticRegression(C=0.4))])
pipe=[pipe_beauty,pipe_mobile,pipe_fashion]


groups=['beauty_','mobile_','fashion']
predictions_val={} # Use for ensembling model
predictions_proba_val={} # Use for ensembling model
predictions_all={} # check in submission file 
predictions_proba_all={} # Use for final ensembling
for i in tqdm(range(3)):
    print("Results for ", groups[i])
    model=pipe[i]
    X=train[train.Group==groups[i]].title
    y=train[train.Group==groups[i]].Category
    model.fit(X,y)
    predictions_val[groups[i]]=model.predict(test[test.Group==groups[i]].title)
    predictions_proba_val[groups[i]]=model.predict_proba(test[test.Group==groups[i]].title)
    print('Accuracy  ')
    print(accuracy_score(test[test.Group==groups[i]].Category,predictions_val[groups[i]]))

    
    model=pipe[i]
    X=df_train[df_train.Group==groups[i]].title
    y=df_train[df_train.Group==groups[i]].Category
    model.fit(X,y)
    predictions_all[groups[i]]=model.predict(df_test[df_test.Group==groups[i]].title)
    predictions_proba_all[groups[i]]=model.predict_proba(df_test[df_test.Group==groups[i]].title)
    


## Wow, interesting ! It turns out that stop words helps only for fashion category!
## Lets' list CV results for beauty, mobile  and fashion 
## beauty CV 0.784223069756
## Mobile CV 0.825934011102
## fashion CV 0.644946336401 with stop words

# Results with stop words.
# {'beauty_': 0.78344144628257784,
#  'mobile_': 0.8236574564959771,
#  'fashion': 0.6449463364011252}

# Our base model CV results. I should list them here



  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Results for  beauty_
Accuracy  
0.789800582724



 33%|███████████████████████████▋                                                       | 1/3 [08:48<17:36, 528.28s/it]

Results for  mobile_
Accuracy  
0.830687665679



 67%|███████████████████████████████████████████████████████▎                           | 2/3 [12:45<07:21, 441.03s/it]

Results for  fashion
Accuracy  
0.652056166223



100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [27:02<00:00, 565.79s/it]

In [53]:
classes={}

for i in range(3):
    classes[groups[i]]=pipe[i].classes_

In [54]:
classes

{'beauty_': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16], dtype=int64),
 'mobile_': array([31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57], dtype=int64),
 'fashion': array([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], dtype=int64)}

In [55]:
predictions_all['beauty_'].shape

(76545,)

In [56]:
predictions_val['beauty_'].shape

(57317,)

In [57]:
# Save all data both validation set and training set
# This data will be used for modeling ensembling and for final ensemble result
for i in range(3):
    # for ensembling
    df=pd.DataFrame(predictions_proba_all[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=df_test[df_test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_all_data2.csv',index=False)
    # for ensemble modeling
    df=pd.DataFrame(predictions_proba_val[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=test[test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_val_data2.csv',index=False)


In [61]:
def submission(name,dic):
    df_sub=df_test.copy()
    for group in dic.keys():
        df_sub.loc[df_sub.Group==group,'Category']=dic[group]
    df_sub.loc[:,'Category']=df_sub.Category.astype('int')
    df_sub[['itemid','Category']].to_csv(name,index=False)

In [62]:

#LB: 0.76272 , previous score LB: 0.76140
submission('2nd_nlp_optimized.csv',predictions_all)

## Let's do ensembling now! 
## So we have results from:
## 1) CountVectorization with logistic regression : lr ending
## 2) CountVectorization with Naive Bayes:  nb ending
## 3) Image categorization:  im ending
## 4) word2vec predictions: w2v ending


## 1 and 2 might be highly correlated with each other. I might remove Naive Bayes. But we also can try with and without and observe results.


In [20]:
df_lr={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_test_proba_nlp_val_data2.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_nlp_all_data2.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_lr[groups[i]]=val_test

In [21]:
df_w2v={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_test_proba_w2v_val_data.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_w2v_all_data.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_w2v[groups[i]]=val_test

In [22]:
df_nb={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_val_proba_nlp_nb.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_nlp_nb.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_nb[groups[i]]=val_test

In [23]:
df_im={}
names=['beauty','mobile','fashion']
for i in range(3):
    val_test=[]
    df1=pd.read_csv('val_'+names[i]+'.csv') # they are ordered very differently
    df2=pd.read_csv('test_'+names[i]+'.csv') # they are ordered very differently
    val_test.append(df1)
    val_test.append(df2)
    df_im[groups[i]]=val_test


In [24]:
# we need to order them according to their itemid number. So, that we con join all of the predictions
df_im['beauty_'][0].head()

,index,0,1,10,11,12,13,14,15,16,2,3,4,5,6,7,8,9,final_pred,Category
0,0057fd944e705684de3b169f1fd7b20e.jpg,0.000922,0.063919,0.000129,0.002412,0.036417,0.015067,0.011914,0.000337,0.017487,0.075449,0.579613,0.063179,0.118592,0.000373,0.002519,0.002103,0.009568,3,0
1,0093bb60e9adb6b2d842d87d4a8d6130.jpg,0.016552,0.028079,0.000746,0.091571,0.003485,0.000700,0.000629,0.000639,0.000780,0.275765,0.210556,0.026311,0.011866,0.011437,0.027535,0.285640,0.007712,8,0
2,00e358a25a00c6edabbff0f7c4fa424e.jpg,0.001821,0.001831,0.000504,0.000466,0.764857,0.050797,0.099181,0.008131,0.030099,0.001431,0.007295,0.022690,0.001862,0.000222,0.004479,0.000819,0.003514,12,0
3,0123b0a5ebbd601a14357274ebcdde46.jpg,0.019760,0.051505,0.002228,0.005470,0.037858,0.023969,0.008131,0.001418,0.019008,0.006417,0.043640,0.609103,0.086127,0.002845,0.019252,0.018621,0.044650,4,0
4,01ec05eb7e3f009b254ccc116d21ed3c.jpg,0.171279,0.184560,0.000596,0.099867,0.001841,0.000397,0.000278,0.000162,0.000261,0.060845,0.159701,0.058774,0.020588,0.052695,0.024520,0.146287,0.017349,1,0


In [25]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_cat=pd.read_json('categories.json')

In [26]:
df_train['Group']=df_train.image_path.map(lambda x: x[:7])
df_test['Group']=df_test.image_path.map(lambda x: x[:7])

In [27]:
df_train['index']=df_train['image_path']
df_test['index']=df_test['image_path']

In [28]:
# remove folder names
df_train.loc[df_train.Group=='beauty_','index']=df_train[df_train.Group=='beauty_']['image_path'].map(lambda x: x[13:])
df_train.loc[df_train.Group=='mobile_','index']=df_train[df_train.Group=='mobile_']['image_path'].map(lambda x: x[13:])
df_train.loc[df_train.Group=='fashion','index']=df_train[df_train.Group=='fashion']['image_path'].map(lambda x: x[14:])

df_test.loc[df_test.Group=='beauty_','index']=df_test[df_test.Group=='beauty_']['image_path'].map(lambda x: x[13:])
df_test.loc[df_test.Group=='mobile_','index']=df_test[df_test.Group=='mobile_']['image_path'].map(lambda x: x[13:])
df_test.loc[df_test.Group=='fashion','index']=df_test[df_test.Group=='fashion']['image_path'].map(lambda x: x[14:])

In [29]:
# Remember that validation set is test set here. We do it for all of the algorithms
train,test=train_test_split(df_train,random_state=2019,stratify=df_train.Category,test_size=0.2)

In [30]:
# Sanity check
for group in groups:
    print(group,test[test.Group==group].shape[0], df_im[group][0].shape[0])

beauty_ 57317 57317
mobile_ 32065 32065
fashion 43941 43941


In [31]:
# Sanity check
for group in groups:
    print(group,df_test[df_test.Group==group].shape[0], df_im[group][1].shape[0])

beauty_ 76545 76545
mobile_ 40417 40417
fashion 55440 55440


In [32]:
df=test[test.Group=='mobile_'][['itemid','index']]

In [33]:
# There is no .jpg extension in the image path of training dataset for fashion let's add '.jpg'
# There is no such problem in df_test
test.loc[test.Group=='fashion','index']=test.loc[test.Group=='fashion','index']+'.jpg'

In [34]:
test.loc[test.Group=='fashion','index'].head()

473036    cb790137111e2525cd895c3375fe84c0.jpg
355772    cbedaa2404fb377dc6c271a1081f7baa.jpg
486345    7a4194bf7b509f29bfc5dcd9cc0355d9.jpg
347206    a15e616c290fb142b1a8b217951f765c.jpg
294868    c343ed06853c285d85996d5302ccf90e.jpg
Name: index, dtype: object

In [35]:
df_test.loc[:,'image_path']=df_test.loc[:,'index']

In [36]:
# This should put everything in order. Something wrong with fashion but 'beauty_' and 'mobile_' works
for group in groups:    
    df=test[test.Group==group][['itemid','index']]
    df_im[group][0]=pd.merge(df,df_im[group][0],on='index')
    df=df_test[df_test.Group==group][['itemid','image_path']]
    df_im[group][1]=pd.merge(df,df_im[group][1],on='image_path')

In [37]:
# Let's check fashion
df_im['fashion'][1].head()

,itemid,image_path,17,18,19,20,21,22,23,24,25,26,27,28,29,30,pred
0,670968580,b0ca9645c5c3527882d935f17b4c9579.jpg,0.008724,0.574313,0.058255,0.032856,0.124793,0.107094,0.002102,0.010663,0.019108,0.040884,0.011683,0.006663,0.001077,0.001785,18
1,475599610,c74af2636e9fe2686981decce3ca5b6c.jpg,0.002250,0.322596,0.074863,0.455304,0.060914,0.046577,0.004295,0.031431,0.000120,0.000969,0.000141,0.000131,0.000200,0.000208,20
2,1652671018,1f2a2d91f5eb2d80f1dfdcf0b1f50169.jpg,0.000405,0.712864,0.031360,0.019982,0.154389,0.031706,0.001268,0.047718,0.000021,0.000043,0.000007,0.000173,0.000003,0.000061,18
3,1705634904,fdb6309635ebb5eac5df9c61984ffce5.jpg,0.004735,0.422191,0.034109,0.056827,0.072294,0.330482,0.001078,0.003901,0.032480,0.020758,0.004922,0.011161,0.004175,0.000888,18
4,1789421087,578e5c74e97128cd25378ef0f0c021cf.jpg,0.044444,0.232101,0.073508,0.034555,0.050782,0.053765,0.007527,0.004144,0.069842,0.335055,0.066570,0.011377,0.011835,0.004495,26


In [38]:
df_test[df_test.Group=='fashion'].head()

,itemid,title,image_path,Group,index
76545,670968580,women casual o neck long sleeve lace patchwork...,b0ca9645c5c3527882d935f17b4c9579.jpg,fashion,b0ca9645c5c3527882d935f17b4c9579.jpg
76546,475599610,zaful long dress vintage wanita v neck lengan ...,c74af2636e9fe2686981decce3ca5b6c.jpg,fashion,c74af2636e9fe2686981decce3ca5b6c.jpg
76547,1652671018,dress skater swing o neck tanpa lengan desain ...,1f2a2d91f5eb2d80f1dfdcf0b1f50169.jpg,fashion,1f2a2d91f5eb2d80f1dfdcf0b1f50169.jpg
76548,1705634904,ok dress slim sexy deep v neck lengan panjang ...,fdb6309635ebb5eac5df9c61984ffce5.jpg,fashion,fdb6309635ebb5eac5df9c61984ffce5.jpg
76549,1789421087,noa korean lace embroidery womens crew neck lo...,578e5c74e97128cd25378ef0f0c021cf.jpg,fashion,578e5c74e97128cd25378ef0f0c021cf.jpg


In [39]:
# Now let's save in proper format and arrangment. So we don't need to spend time on redoing samething for ensembling

for group in groups:
    df_im[group][0].to_csv(group+'_val_im.csv',index=False)
for group in groups:
    df_im[group][1].to_csv(group+'_test_im.csv',index=False)

# finally, we can start ensembling. Let's first check their relative error in internal test set. Also, correlations btw them

In [40]:
results=[df_lr,df_nb,df_w2v,df_im]

In [41]:
# Let's define columns for each category
columns={}
columns['beauty_']=[str(i) for i in range(17)]
columns['mobile_']=[str(i) for i in range(31,58)]
columns['fashion']=[str(i) for i in range(17,31)]

In [42]:
models={'lr':df_lr,'nb':df_nb,'w2v':df_w2v,'im':df_im}

In [43]:
for name, model in models.items():
    if name=='im':
        pred='final_pred'
    else:
        pred='pred'
    print('Accuracy score for model: ', name)
    for group in groups:
        print(group,accuracy_score(test[test.Group==group].Category,model[group][0][pred]))

Accuracy score for model:  lr
beauty_ 0.789800582724
mobile_ 0.830687665679
fashion 0.652056166223
Accuracy score for model:  nb
beauty_ 0.764188635134
mobile_ 0.778949009824
fashion 0.592362486061
Accuracy score for model:  w2v
beauty_ 0.744054992411
mobile_ 0.766505535631
fashion 0.579777428825
Accuracy score for model:  im
beauty_ 0.640647626359
mobile_ 0.562763137377
fashion 0.490612412098


In [44]:
df_all={}
for group in groups:
    val_test=[pd.DataFrame(),pd.DataFrame()]
    df_all[group]=val_test

In [45]:
df1=models['lr']['beauty_'][0][columns['beauty_']].copy()

In [46]:
df2=models['nb']['beauty_'][0][columns['beauty_']].copy()

In [47]:
df1.join(df2,rsuffix='_nb',lsuffix='_lr' )

,0_lr,1_lr,2_lr,3_lr,4_lr,5_lr,6_lr,7_lr,8_lr,9_lr,...,7_nb,8_nb,9_nb,10_nb,11_nb,12_nb,13_nb,14_nb,15_nb,16_nb
0,0.000054,0.891675,0.000082,0.000013,0.002232,0.104553,5.695655e-06,0.000581,5.566095e-05,0.000225,...,3.822854e-70,3.247129e-75,5.846333e-68,1.504201e-72,3.091104e-74,1.073869e-74,1.267185e-75,1.082068e-75,4.888542e-76,1.786372e-74
1,0.001044,0.006834,0.003689,0.956227,0.012288,0.002244,6.158536e-04,0.006582,2.519976e-03,0.003677,...,4.692910e-08,4.352797e-09,1.323928e-08,1.775783e-10,6.762142e-09,1.189405e-09,2.179270e-10,2.054399e-10,5.195402e-11,1.698754e-10
2,0.001438,0.055271,0.513170,0.355519,0.017127,0.041664,1.167639e-03,0.006082,1.678843e-03,0.001938,...,8.616342e-03,5.522715e-03,6.537988e-03,1.104245e-03,3.763031e-03,1.284933e-02,2.878211e-03,2.676073e-03,6.296645e-04,2.196094e-03
3,0.000254,0.038222,0.000271,0.015969,0.002330,0.921653,1.065826e-05,0.001378,3.473787e-04,0.012019,...,3.017239e-29,2.980085e-33,1.823110e-27,1.111854e-36,6.466593e-26,2.859992e-31,7.007362e-36,3.339887e-35,2.069782e-37,1.375392e-33
4,0.000330,0.050132,0.000204,0.002160,0.011916,0.016006,1.291002e-03,0.000277,2.415868e-04,0.916303,...,4.506240e-53,1.285230e-53,1.000000e+00,1.995092e-54,2.988064e-54,9.702951e-54,3.681020e-53,5.508834e-53,1.374844e-54,3.883131e-53
5,0.001460,0.956315,0.006673,0.001208,0.028033,0.001397,4.453819e-04,0.000891,7.752054e-04,0.002240,...,4.492726e-31,1.673633e-30,7.967078e-31,3.247524e-33,5.836608e-32,6.580851e-33,5.447095e-33,5.604401e-33,2.237911e-33,2.917288e-32
6,0.000057,0.003863,0.000386,0.105864,0.008115,0.878426,1.331391e-05,0.001411,8.432158e-05,0.000076,...,1.680966e-38,2.798178e-42,5.202824e-39,2.301987e-43,1.330024e-41,1.067549e-43,1.168693e-37,4.148950e-42,1.714114e-43,1.841132e-42
7,0.000226,0.191250,0.000679,0.746496,0.040365,0.018406,1.147274e-05,0.001555,5.066422e-06,0.000031,...,1.258494e-25,2.775293e-31,1.980863e-25,6.028844e-31,2.579815e-25,2.924825e-32,2.333343e-31,2.445348e-31,1.075025e-31,2.143038e-31
8,0.002326,0.047553,0.002027,0.019117,0.031114,0.871940,9.813073e-04,0.011639,6.182607e-03,0.003985,...,4.933126e-06,7.680213e-08,3.942346e-07,1.625322e-09,3.525294e-08,4.115906e-08,4.098537e-11,5.151592e-11,3.582684e-11,6.389668e-11
9,0.000911,0.000312,0.005487,0.961239,0.015146,0.005119,2.021340e-05,0.000109,1.209585e-05,0.006396,...,2.957403e-32,3.604432e-33,6.748749e-24,3.167339e-34,5.577824e-32,1.668348e-28,2.544986e-33,6.280499e-34,5.049852e-35,1.597852e-33


In [48]:
len(models.keys())

4

In [49]:
#  Let's put all of the predictions together for ensembling
# I am still not sure about best stragery for ensembling
df_all_val={}
df_all_test={}
for group in groups:
    df_all_val[group]=models['lr'][group][0][columns[group]].copy()
    df_all_test[group]=models['lr'][group][1][columns[group]].copy()
    for name, model in models.items():
        if name!='lr':
            df1=models[name][group][0][columns[group]].copy()
            df_all_val[group]=df_all_val[group].join(df1,rsuffix=name)
            
            df2=models[name][group][1][columns[group]].copy()
            df_all_test[group]=df_all_test[group].join(df2,rsuffix=name)
        
# Now we put everything together.

In [50]:
# Sanity check
for group in groups:
    print("shape of df_all_val and test", group)
    print(df_all_val[group].shape, test[test.Group==group].shape)

shape of df_all_val and test beauty_
(57317, 68) (57317, 6)
shape of df_all_val and test mobile_
(32065, 108) (32065, 6)
shape of df_all_val and test fashion
(43941, 56) (43941, 6)


In [51]:
for group in groups:
    print("shape of df_all_test and df_test", group)
    print(df_all_test[group].shape, df_test[df_test.Group==group].shape)

shape of df_all_test and df_test beauty_
(76545, 68) (76545, 5)
shape of df_all_test and df_test mobile_
(40417, 108) (40417, 5)
shape of df_all_test and df_test fashion
(55440, 56) (55440, 5)


In [52]:
from sklearn.neighbors import KNeighborsClassifier

In [53]:
lr=LogisticRegression()
svm=SVC(kernel='linear')
rf=RandomForestClassifier(n_estimators=200)
knn=KNeighborsClassifier(n_neighbors=50)

In [54]:
pipe_en={}
for group in groups:
    pipe_en[group]=Pipeline([('model',lr)])

In [55]:
param={'model':[lr,rf,knn,svm]}
best_scores={}
best_param={}
for group in tqdm(groups):
    print('Results for ', group)
    grid_search=GridSearchCV(pipe_en[group],param_grid=param,cv=5)
    y=test[test.Group==group].Category.values
    X=df_all_val[group]
    grid_search.fit(X,y)
    print("Best score:", grid_search.best_score_," for "+group)
    print("Best params:", grid_search.best_params_," for "+group)
    best_scores[group]=grid_search.best_score_
    best_param[group]=grid_search.best_params_
    
    

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Results for  beauty_
Best score: 0.790672924263  for beauty_
Best params: {'model': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)}  for beauty_


 33%|███████████████████████████▎                                                      | 1/3 [25:23<50:46, 1523.49s/it]

Results for  mobile_
Best score: 0.831124278809  for mobile_
Best params: {'model': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)}  for mobile_


 67%|██████████████████████████████████████████████████████▋                           | 2/3 [39:31<22:00, 1320.99s/it]

Results for  fashion
Best score: 0.663389545072  for fashion
Best params: {'model': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)}  for fashion


100%|██████████████████████████████████████████████████████████████████████████████████| 3/3 [57:00<00:00, 1239.10s/it]


In [56]:
pipe_en2={}
for group in groups:
    pipe_en2[group]=Pipeline([('model',rf)])

In [57]:
param={'model__n_estimators':[i for i in range(100,300,20)]}
best_scores={}
best_param={}
for group in tqdm(groups):
    print('Results for ', group)
    grid_search=GridSearchCV(pipe_en2[group],param_grid=param,cv=5)
    y=test[test.Group==group].Category.values
    X=df_all_val[group]
    grid_search.fit(X,y)
    print("Best score:", grid_search.best_score_," for "+group)
    print("Best params:", grid_search.best_params_," for "+group)
    best_scores[group]=grid_search.best_score_
    best_param[group]=grid_search.best_params_

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Results for  beauty_
Best score: 0.791649946787  for beauty_
Best params: {'model__n_estimators': 280}  for beauty_


 33%|██████████████████████████                                                    | 1/3 [1:02:35<2:05:10, 3755.35s/it]

Results for  mobile_
Best score: 0.831155465461  for mobile_
Best params: {'model__n_estimators': 140}  for mobile_


 67%|█████████████████████████████████████████████████████▎                          | 2/3 [1:49:41<57:56, 3476.58s/it]

Results for  fashion
Best score: 0.663890216427  for fashion
Best params: {'model__n_estimators': 160}  for fashion


100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [2:23:27<00:00, 3041.43s/it]


In [60]:
# Now let' make our first submission
# LB: 0.76413

pipe_beauty=Pipeline([('model',RandomForestClassifier(n_estimators=280))]) # we might try to increase this number since it is still improving
pipe_mobile=Pipeline([('model',RandomForestClassifier(n_estimators=140))])
pipe_fashion=Pipeline([('model',RandomForestClassifier(n_estimators=160))])

pipe_en2={'beauty_':pipe_beauty,'mobile_':pipe_mobile,'fashion':pipe_fashion}
predictions={}
for group in tqdm(groups):
    print('Results for ', group)
    model=pipe_en2[group]
    y=test[test.Group==group].Category.values
    X=df_all_val[group]
    model.fit(X,y)
    X_test=df_all_test[group]
    y_test=model.predict(X_test)
    predictions[group]=y_test





  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

Results for  beauty_


 33%|███████████████████████████▋                                                       | 1/3 [02:19<04:38, 139.40s/it]

Results for  mobile_


 67%|███████████████████████████████████████████████████████▎                           | 2/3 [03:13<01:53, 113.93s/it]

Results for  fashion


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:58<00:00, 93.08s/it]


In [72]:
df_sub=df_test[['itemid','Group']].copy()
df_sub['Category']=0
for group in groups:
    df_sub.loc[df_sub.Group==group,'Category']=predictions[group]
del df_sub['Group']
df_sub.to_csv('ensemble_2.csv',index=False)

In [83]:
from sklearn.model_selection import StratifiedKFold

In [84]:
# I am not sure but we might be overfitting. Let's try our previous approach


def eval_model():
    folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    predictions2={}
    for group in groups:
        print("Results for group", group)
        oof=np.zeros(len(df_all_val[group]))
        pred=np.zeros((len(df_test[df_test.Group==group]),folds.n_splits))
        X=df_all_val[group].values
        y=test[test.Group==group].Category.values
        X_sub=df_all_test[group].values
        model=pipe_en2[group]
        for fold_,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
            print("Fold:",fold_)
            X_train=X[trn_idx]
            y_train=y[trn_idx]
            model.fit(X_train,y_train)
            X_val=X[val_idx]
            oof[val_idx]=model.predict(X_val)
            pred[:,fold_]=model.predict(X_sub)
            print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
        print('Acuracy of overall:',accuracy_score(y,oof))
        predictions2[group]=pred
    return predictions2


In [85]:
pred=eval_model()

Results for group beauty_
Fold: 0
Acuracy of fold: 0.78962510898
Fold: 1
Acuracy of fold: 0.788137810728
Fold: 2
Acuracy of fold: 0.799790630725
Fold: 3
Acuracy of fold: 0.791921130693
Fold: 4
Acuracy of fold: 0.787466177883
Acuracy of overall: 0.791388244325
Results for group mobile_
Fold: 0
Acuracy of fold: 0.829648084709
Fold: 1
Acuracy of fold: 0.826242792582
Fold: 2
Acuracy of fold: 0.830059245401
Fold: 3
Acuracy of fold: 0.825300265169
Fold: 4
Acuracy of fold: 0.831276363068
Acuracy of overall: 0.828504600031
Results for group fashion
Fold: 0
Acuracy of fold: 0.666856232939
Fold: 1
Acuracy of fold: 0.666704584234
Fold: 2
Acuracy of fold: 0.662116040956
Fold: 3
Acuracy of fold: 0.660521224536
Fold: 4
Acuracy of fold: 0.655733971074
Acuracy of overall: 0.662388202362


In [104]:
predictions2={}
for group in groups:
    predictions2[group]=pd.DataFrame(pred[group]).mode(axis=1)[0].astype('int').values

In [105]:
for group in groups:
    print(df_test[df_test.Group==group].shape,predictions2[group].shape)

(76545, 5) (76545,)
(40417, 5) (40417,)
(55440, 5) (55440,)


In [106]:
# LB:0.76428  just slightly better than previous
df_sub=df_test[['itemid','Group']].copy()
df_sub['Category']=0
for group in groups:
    print(group)
    df_sub.loc[df_sub.Group==group,'Category']=predictions2[group]
del df_sub['Group']
df_sub.to_csv('ensemble_2_v2.csv',index=False)

beauty_
mobile_
fashion


In [ ]:
#  Lets try some sort of averaging predict probas.


In [100]:
df_sub[df_sub.Group=='beauty_']

,itemid,Group,Category
0,370855998,beauty_,5
1,637234604,beauty_,5
2,690282890,beauty_,5
3,930913462,beauty_,4
4,1039280071,beauty_,5
5,1327710392,beauty_,3
6,1328802799,beauty_,4
7,1330468145,beauty_,5
8,1677309730,beauty_,5
9,1683142205,beauty_,5


In [68]:
pipe_en=Pipeline([('model',lr)])

In [ ]:
# Previous function to evaluate model for beauty products
def eval_model_tr(model):
    y=df_beauty_val.Category
    folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    oof=np.zeros(len(df_all_prob))
    # prediction for the model with 80 of data
#     pred_80=np.zeros((len(X_test_beauty1),folds.n_splits))
    #prediction for the model with 100 of data
#     pred_100=np.zeros((len(X_test_beauty2),folds.n_splits))

    for fold_,(trn_idx,val_idx) in enumerate(folds.split(df_all_prob,y)):
#         print("Fold:",fold_)
        X_train=df_all_prob.loc[trn_idx,:]
        y_train=y[trn_idx]
        model.fit(X_train,y_train)
        X_val=df_all_prob.loc[val_idx,:]
        oof[val_idx]=model.predict(X_val)
#         pred_80[:,fold_]=model.predict(X_test_beauty1)
#         pred_100[:,fold_]=model.predict(X_test_beauty2)
#         print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
    print('Acuracy of overall:',accuracy_score(y,oof))
    return accuracy_score(y,oof)

In [294]:
pd.merge(df,df_im['fashion'][0],on='index')

,itemid,index,31,32,33,34,35,36,37,38,...,50,51,52,53,54,55,56,57,final_pred,Category


In [219]:
#Let's check if it works
df_im['mobile_'][0].head()

,itemid,index,31,32,33,34,35,36,37,38,...,50,51,52,53,54,55,56,57,final_pred,Category
0,1058517290,3912586cdf51144db783b2bac1ee4d3f.jpg,0.026980,0.020451,0.004094,0.407791,0.372147,0.000012,0.001650,0.001870,...,0.005001,0.000660,0.001180,0.000943,0.000728,0.000967,0.000833,0.000563,34,34
1,1195506745,bb327fec3989b18f714deb3c07203fb8.jpg,0.072345,0.246230,0.057411,0.097349,0.273988,0.002055,0.027128,0.079219,...,0.000813,0.000252,0.000367,0.000301,0.000122,0.001252,0.000214,0.000445,35,35
2,1833821649,8959112348969556df29f24d06ecff8f.jpg,0.092200,0.167811,0.000182,0.012845,0.275278,0.000306,0.001374,0.000576,...,0.000136,0.002763,0.001903,0.000443,0.007212,0.000416,0.000406,0.000569,42,35
3,291297752,8733bba77a727e4771d51b5a8b48c8a5.jpg,0.043451,0.026404,0.017773,0.651762,0.116616,0.000137,0.004889,0.001703,...,0.000309,0.000054,0.000225,0.000511,0.000033,0.000112,0.000249,0.000087,34,34
4,1812460647,af7047829e497014daa5c2b4914a0fb4.jpg,0.872700,0.008431,0.000067,0.021418,0.066922,0.000281,0.000757,0.000065,...,0.000005,0.000052,0.000018,0.000056,0.000236,0.000021,0.000038,0.000015,31,31


In [220]:
df_train[df_train.Group=='mobile_'].head()

,itemid,title,Category,image_path,Group,index
506285,2346660,apple iphone 4s back glass spare part original...,31,mobile_image/a9c8f0fdd6587deed197634066cf7eee.jpg,mobile_,a9c8f0fdd6587deed197634066cf7eee.jpg
506286,2816338,iphone 4s 64gb white,31,mobile_image/3b9a11608551b11b9330268e0d055e01.jpg,mobile_,3b9a11608551b11b9330268e0d055e01.jpg
506287,2847602,samsung sm b310e piton dual sim,32,mobile_image/1d719e936841a83c165da620f927de68.jpg,mobile_,1d719e936841a83c165da620f927de68.jpg
506288,3116949,samsung caramel gt e1272 dual sim 32 mb putih,32,mobile_image/1d35a74d90df6cf4a02e6a5df9e9ff29.jpg,mobile_,1d35a74d90df6cf4a02e6a5df9e9ff29.jpg
506289,3794648,garskin sony experia z z1 z2 ultra,33,mobile_image/5556577b09539a9c0db0d00e0f171e2d.jpg,mobile_,5556577b09539a9c0db0d00e0f171e2d.jpg


In [139]:
?pd.merge

In [119]:
df.merge(df_im['mobile_'][0],on='index')

,itemid,index,31,32,33,34,35,36,37,38,...,50,51,52,53,54,55,56,57,final_pred,Category
0,2816338,3b9a11608551b11b9330268e0d055e01.jpg,0.883424,0.012605,0.000212,0.016350,0.073758,0.000144,0.000176,0.000084,...,2.546633e-05,1.471064e-05,0.000021,0.000019,0.000057,0.000040,0.000084,1.323019e-05,31,31
1,9503620,a23f0381039e5595559be27db3271d2f.jpg,0.278721,0.177533,0.003252,0.070696,0.063292,0.001435,0.000578,0.001302,...,3.675401e-04,1.966892e-04,0.000595,0.000358,0.000317,0.000214,0.001195,1.328025e-04,41,31
2,26702291,2c0ea46a95837e5171bd43512e82b1ce.jpg,0.001377,0.978527,0.000005,0.003402,0.013050,0.000007,0.000013,0.000019,...,2.433453e-07,1.700982e-06,0.000048,0.000004,0.000006,0.000010,0.000034,1.335207e-06,32,32
3,74050515,9a28a64c29e97e52c71ed45b029281ec.jpg,0.601693,0.033724,0.028193,0.028183,0.180513,0.002939,0.007077,0.011178,...,2.531332e-03,1.450678e-04,0.000201,0.000203,0.000186,0.000221,0.000318,3.438597e-04,31,31
4,122874123,3cfd98eb1f103623e12b8f081d702a66.jpg,0.048917,0.014367,0.001691,0.500844,0.248733,0.000180,0.003880,0.000100,...,7.084288e-05,2.756204e-05,0.000732,0.000258,0.000125,0.000090,0.000650,1.623501e-04,34,34
5,148667803,083faffd864dd1e6aff8cbc892ac15d7.jpg,0.623251,0.154122,0.004254,0.039050,0.051524,0.000153,0.003944,0.000892,...,1.371836e-05,4.439028e-04,0.000025,0.000118,0.000064,0.000020,0.000081,3.032195e-05,31,31
6,193655696,c1b8da227c7d1d4a4edd84496e914771.jpg,0.030702,0.029294,0.002633,0.014957,0.078660,0.802044,0.003124,0.009234,...,7.922183e-04,1.047876e-04,0.000475,0.002041,0.000055,0.000551,0.000276,1.658877e-04,36,36
7,210801578,f14091a8b8a09cd6b5e64b6ae2b45185.jpg,0.277034,0.214561,0.084110,0.074292,0.241911,0.001167,0.019214,0.004437,...,8.234581e-05,8.576331e-05,0.000280,0.003183,0.000413,0.000417,0.000455,1.556106e-04,31,32
8,212660404,39ed4573ea35e9b4e4619f37734c512a.jpg,0.014043,0.899937,0.000496,0.012193,0.026526,0.000140,0.000853,0.005278,...,2.121018e-05,4.890963e-04,0.000072,0.000279,0.000517,0.000105,0.000717,5.743321e-05,32,32
9,213418407,d961511fb22ca5eec825219035c0e1d4.jpg,0.005258,0.458902,0.001525,0.233880,0.271702,0.000182,0.001247,0.002195,...,3.200407e-05,4.720400e-04,0.000090,0.002059,0.000435,0.000022,0.000331,8.762698e-05,32,32


In [ ]:
for i in range(3):
    df=pd.DataFrame(predictions_proba[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=df_test[df_test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_all_data.csv',index=False)

In [72]:
y_pred=pipe.predict(X_train_EN)

In [74]:
accuracy_score(y_train,y_pred)

0.70672370657017336

In [ ]:
vectorizerEN = Word2VecVectorizerEN()

for group in groups:
    # English vectorization
    print('Results for group: ', group)
    vectorizerEN = Word2VecVectorizerEN()
    X_train_EN = vectorizerEN.fit_transform(train[train.Group==group].title)
    X_test_EN = vectorizerEN.transform(train[train.Group==group].title)
    
    # Indonesian vectorization
    vectorizerID = Word2VecVectorizerID()
    X_train_ID = vectorizerID.fit_transform(X_beauty_train)
    y_train = train[train.Group==group].Category.values
    X_test_ID = vectorizerID.transform(X_beauty_test)
    y_test = test[test.Group==group].Category.values
    
    # Combination of two models
    df_train_combined=pd.DataFrame(X_train_EN)
    df_train_combined=df_train_combined.join(pd.DataFrame(X_train_ID),rsuffix='ID')
    df_test_combined=pd.DataFrame(X_test_EN)
    df_test_combined=df_test_combined.join(pd.DataFrame(X_test_ID),rsuffix='ID')
    X_train_com=df_train_combined.values
    X_test_com=df_test_combined.values
    
    



    

In [81]:
#vectorizer = GloveVectorizer()
vectorizerEN = Word2VecVectorizerEN()
X_train_EN = vectorizerEN.fit_transform(X_beauty_train)
#Ytrain = y_beauty_train

X_test_EN = vectorizerEN.transform(X_beauty_test)
#Ytest = y_beauty_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 1639 / 200608
Numer of samples with no words found: 680 / 85975


In [82]:
X_train_EN.shape

(200608, 300)

In [83]:
#vectorizer = GloveVectorizer()
vectorizerID = Word2VecVectorizerID()
X_train_ID = vectorizerID.fit_transform(X_beauty_train)
y_train = y_beauty_train

X_test_ID = vectorizerID.transform(X_beauty_test)
y_test = y_beauty_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 164061 / 200608
Numer of samples with no words found: 70373 / 85975


In [84]:
X_train_ID.shape

(200608, 300)

In [96]:
df_train_combined=pd.DataFrame(X_train_EN)

In [99]:
df_train_combined=df_train_combined.join(pd.DataFrame(X_train_ID),rsuffix='ID')

In [100]:
df_test_combined=pd.DataFrame(X_test_EN)
df_test_combined=df_test_combined.join(pd.DataFrame(X_test_ID),rsuffix='ID')

In [103]:
df_train_combined.shape,df_test_combined.shape

((200608, 600), (85975, 600))

In [105]:
model = RandomForestClassifier(n_estimators=200)
model.fit(df_train_combined.values,y_train.values)
print("train score", model.score(df_train_combined.values,y_train.values))
print("test score:", model.score(df_test_combined.values,y_test.values))

train score 0.898877412665
test score: 0.730386740331


# Let's do the same for fashion 

In [106]:
#vectorizer = GloveVectorizer()
vectorizerID = Word2VecVectorizerID()
X_train_ID = vectorizerID.fit_transform(X_fashion_train)
y_train = y_fashion_train

X_test_ID = vectorizerID.transform(X_fashion_test)
y_test = y_fashion_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 122720 / 153791
Numer of samples with no words found: 52577 / 65911


In [107]:
#vectorizer = GloveVectorizer()
vectorizerEN = Word2VecVectorizerEN()
X_train_EN = vectorizerEN.fit_transform(X_fashion_train)
#Ytrain = y_beauty_train

X_test_EN = vectorizerEN.transform(X_fashion_test)
#Ytest = y_beauty_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 144 / 153791
Numer of samples with no words found: 68 / 65911


In [108]:
X_train_EN.shape

(153791, 300)

In [109]:
X_train_ID.shape

(153791, 300)

In [110]:
df_train_combined=pd.DataFrame(X_train_EN)

In [111]:
df_train_combined=df_train_combined.join(pd.DataFrame(X_train_ID),rsuffix='ID')

In [112]:
df_test_combined=pd.DataFrame(X_test_EN)
df_test_combined=df_test_combined.join(pd.DataFrame(X_test_ID),rsuffix='ID')

In [113]:
df_train_combined.shape,df_test_combined.shape

((153791, 600), (65911, 600))

In [114]:
model = RandomForestClassifier(n_estimators=200)
model.fit(df_train_combined.values,y_train.values)
print("train score", model.score(df_train_combined.values,y_train.values))
print("test score:", model.score(df_test_combined.values,y_test.values))

train score 0.928266283463
test score: 0.560968578841


In [15]:
model = RandomForestClassifier(n_estimators=200)

In [16]:
model.fit(Xtrain,Ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
print("train score", model.score(Xtrain,Ytrain))
print("test score:", model.score(Xtest,Ytest))

train score 0.896150701866
test score: 0.731177667927


In [18]:
predictions = model.predict(Xtest)

In [19]:
from sklearn.metrics import classification_report,accuracy_score

In [20]:
print(classification_report(predictions,Ytest))

              precision    recall  f1-score   support

           0       0.30      0.60      0.40       585
           1       0.59      0.76      0.66      6726
           2       0.79      0.85      0.82      3282
           3       0.88      0.80      0.84     26695
           4       0.74      0.61      0.67     15794
           5       0.75      0.75      0.75     16427
           6       0.33      0.43      0.37       480
           7       0.70      0.74      0.72      3265
           8       0.51      0.59      0.54      1574
           9       0.37      0.72      0.49      1268
          10       0.44      0.59      0.50       246
          11       0.38      0.55      0.45       849
          12       0.85      0.73      0.79      7672
          13       0.43      0.63      0.51       588
          14       0.14      0.40      0.21       295
          15       0.12      0.43      0.19        47
          16       0.09      0.34      0.14       182

   micro avg       0.73   

In [21]:
print(accuracy_score(predictions,Ytest))

0.731177667927


In [ ]:
#from sklearn.linear_model import LogisticRegression

In [ ]:
#lr = LogisticRegression()

In [ ]:
#lr.fit(Xtrain,Ytrain)

In [ ]:
#predictions_lr = lr.predict(Xtest)

In [ ]:
#print(accuracy_score(predictions_lr,Ytest))